# Capstone  Week 3 - Segmenting and Clustering Neighborhoods in Toronto

## <font color=red>All 3 parts are included in this Notebook</font> 


## Table of Contents

1. <a href="#item1"><em>Part 1</em> - Web Scraping into Dataframe</a>    
2. <a href="#item2"> <em>Part 2</em> - Getting the Coordinates</a> 
3. <a href="#item3"> <em>Part 3</em> - Neighnourhood Clustering</a>

<a id='item1'></a>
## <font color=blue>Part 1 - Web Scraping into Dataframe</font> 
 

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that - M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the - neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
#Import libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests


#### *Scrape the List of postal codes of Canada*

In [2]:
#with open('C:/Users/handa/OneDrive/Desktop/NWU/Courersa/postcode.html',encoding="utf-8") as fp:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data,'lxml')
# print(soup.prettify())

#### Iteration loop to create df

In [3]:
postcode = []
borough = []
neighborhood = []
n = 0
for row in soup.find('table').find_all('tr'):
    for element in row.find_all('td'):
        records = element.p.find('span')

        record  = re.split( r'\((.*)\)',records.text)
        
        postcode.append(element.p.b.text)
        borough.append(record[0].replace('/', ','))
        
# if the neighborhood is empty, fill it with Borough
        if len(record)>1:
            neighborhood.append(record[1].replace('/', ','))
        else: 
            neighborhood.append(record[0].replace('/', ','))

In [4]:
import pandas as pd
toronto_df = pd.DataFrame({"PostalCode": postcode,
                           "Borough": borough,
                           "Neighborhood": neighborhood})

print(toronto_df.shape)
toronto_df.head()

(180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


#### Remove "Not Assigned"

In [5]:
toronto_df = toronto_df[toronto_df.Borough != 'Not assigned']
print(toronto_df.shape)
toronto_df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government"


<a id='item2'></a>
## <font color=blue> Part 2 - Getting the Coordinates</font> 


load location of csv file

In [6]:
url='http://cocl.us/Geospatial_data'
df_pcodes=pd.read_csv(url)
df_pcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename Postal Code to PostalCode

In [7]:
df_pcodes.columns = ['PostalCode', 'Latitude', 'Longitude']
df_pcodes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the dataframe

In [8]:
df=pd.merge(toronto_df,df_pcodes, how='right', on = 'PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.662301,-79.389494


<a id='item3'></a>
## <font color=blue> Part 3 - Neighbourhood Clustering</font> 


The central loction of Toronto

In [9]:
latitude = 43.6532
longitude= -79.3832

Create map of Toronto using latitude and longitude values

In [10]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

##### The analysis of this part will focus on the Borough containing the work "Toronto"

In [11]:
toronto_df_new= df[df['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Updated Map

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

##### Explore the neighborhoods and segment them using Foursquare API

Let's get the top 100 venues that are within a radius of 500 meters.

In [13]:
# define Foursquare Credentials and Version
CLIENT_ID = 'YGALF3JQ1JU10XRU2P0NT2QH22IQYQZQVED4UTE4BXD2MUI5' 
CLIENT_SECRET = '40I31SU0W3JKROENXU3Z5JMXAKNO1YOIXS0OO4AFQQGZGF0L' 
VERSION = '20180605'

In [14]:
import json

radius = 500
LIMIT = 100

venues = []

for lat, lon, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

Convert the venues list into a new DataFrame

In [15]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1677, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Venues count returned for each PostalCode

In [16]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4J,East YorkEast Toronto,The Danforth East,3,3,3,3,3,3
M4K,East Toronto,"The Danforth West , Riverdale",41,41,41,41,41,41
M4L,East Toronto,"India Bazaar , The Beaches West",17,17,17,17,17,17
M4M,East Toronto,Studio District,41,41,41,41,41,41
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,7,7,7,7,7,7
M4R,Central Toronto,North Toronto West,20,20,20,20,20,20
M4S,Central Toronto,Davisville,33,33,33,33,33,33


Unique Categories

In [17]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 237 uniques categories.


##### Analyze each neighbourhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
fixed_columns = list(toronto_onehot.columns[-2:]) + list(toronto_onehot.columns[:-2])
toronto_onehot = toronto_onehot[fixed_columns]

## move neighborhood column to the first column
cols=list(toronto_onehot.columns.values)
cols.pop(cols.index('Neighborhood'))
toronto_onehot=toronto_onehot[['Neighborhood']+cols]
#toronto_onehot
## rename Neighborhood for Neighbourhood so that future merge works
##toronto_onehot.rename(columns = {'Neighborhood': 'Neighbourhood'}, inplace = True)


print(toronto_onehot.shape)
toronto_onehot.head()

(1677, 239)


,Neighborhood,PostalCode,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park , Harbourfront",M5A,Downtown Toronto,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",M5A,Downtown Toronto,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park , Harbourfront",M5A,Downtown Toronto,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park , Harbourfront",M5A,Downtown Toronto,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park , Harbourfront",M5A,Downtown Toronto,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [19]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood']).mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(39, 237)


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.018182,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,"CN Tower , King and Spadina , Railway Lands , ...",0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,Central Bay Street,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.0,...,0.0,0.0,0.012195,0.0,0.0,0.012195,0.0,0.0,0.0,0.012195
4,Christie,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


Display the top 10 venues for each neighborhood.

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, 1:]#.iloc[1:]
    
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    neighborhoods_venues_sorted.iloc[ind, 1:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

(39, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Cheese Shop,Restaurant,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Shopping Mall
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Breakfast Spot,Café,Furniture / Home Store,Stadium,Intersection,Bar,Bakery,Restaurant,Climbing Gym
2,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
3,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Middle Eastern Restaurant,Japanese Restaurant,Burger Joint,Thai Restaurant,Salad Place,Bubble Tea Shop
4,Christie,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Candy Store,Restaurant,Baby Store,Athletics & Sports,Gas Station


##### Cluster Neighbourhoods

Run k-means to cluster the neighbourhood into 5 clusters.

In [21]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_group_clustering = toronto_grouped.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_clustering)

(kmeans.labels_)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 4, 3, 1,
       3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3])

Cluster as well as the top 10 venues for each neighborhood

In [22]:
toronto_merged = toronto_df_new

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Bakery,Pub,Mexican Restaurant,Theater,Café,Restaurant,French Restaurant,Ice Cream Shop
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Bookstore,Ramen Restaurant,Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Clothing Store,Bakery,Cosmetics Shop,Diner,Beer Bar
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Health Food Store,Pub,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Café,Cheese Shop,Restaurant,Beer Bar,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Shopping Mall


Visualize the clusters in map

In [23]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters